## Get coverage with Mosdepth - Original code

This notebook contains the original scripts shared by Hubert Pausch in order to calculate the coverage of the samples. The goals of these scripts are:

* Generating a coverage file per sample (by using get_coverage.sh) so the coverage of each position within the BAM file is retrieved
* Creating a summary of the coverage where the rows are the samples and the columns the coverage for autosomes and sexual chromosomes.

#### Step 1: get coverage by using mosdepth

This is the script shared by Hubert to calculate the coverage using mosdepth ([paper](https://www.ncbi.nlm.nih.gov/pubmed/29096012) - [Github](https://github.com/brentp/mosdepth)).

In [ ]:
cd $TMPDIR ##Here moving to the root. In my case /cluster/home/avillas

export LD_LIBRARY_PATH=/cluster/work/pausch/group_bin/htslib/ ##Setting the htslib library path (C library for high-throughput sequencing data formats)

file=${path}${bam_id}.bam #path and bam_id will be given as part of the job submission, see cells below.
outfile=${bam_id}.coverage #setting the name of the file generated

for chr in {1..32}
do
if [[ $chr = "30" ]]; then
    chr=X
fi
if [[ $chr = "31" ]]; then
    chr=Y
fi
if [[ $chr = "32" ]]; then
    chr=MT
fi

#The above loop is just setting the names of chromosomes 30, 31 and 32 as X, Y and MT 

/cluster/work/pausch/group_bin/mosdepth -c $chr output $file

#Mosdepth software is run for all bam files.
#-c parameter implies extraction of coverage per chromosome
#output is the prefix needed for the generation of intermediate files (e.g. output.per-base.bed.gz)

if [[ $chr != "X" ]]; then
coverage_sum=`zcat output.per-base.bed.gz | awk '{ total += ($3-$2)*$4; count++ } END {print total}'`;length=`zcat output.per-base.bed.gz | tail -n 1 | awk '{print $3}'`; coverage=`printf %.2f $(echo $coverage_sum/$length | bc -l)`; echo $file $chr $length $coverage >> ${outfile}
fi

#Code run for all chromosomes except for X. Splitting the different functions:
#Decompress output.per-base.bed.gz, see structure of the file below.
#Coverage_sum is the total of coverage for all the positions. This is, sum of multiplying the coverage by the number of positions for such coverage
#total += and count++ are used for increasing the number for the whole file.
#length is the number of end position for the last entry
#coverage is the division between the coverage_sum and the length, with 2 decimals
#finally 4 columns are addded to the output file: file, chromosome, length and coverage

if [[ $chr = "X" ]]; then
coverage_sum=`zcat output.per-base.bed.gz | awk '$2<133300518' | awk '{ total += ($3-$2)*$4; count++ } END {print total}'`;length=133300518; coverage=`printf %.2f $(echo $coverage_sum/$length | bc -l)`; echo $file $chr 133300518 $coverage >> ${outfile}
coverage_sum=`zcat output.per-base.bed.gz | awk '$2>133300518' | awk '{ total += ($3-$2)*$4; count++ } END {print total}'`;length=5708626; coverage=`printf %.2f $(echo $coverage_sum/$length | bc -l)`; echo $file PAR 5708626 $coverage >> ${outfile}
fi

#Code run for X chromosomes. Depending on starting position:
#For positions under 133300518 the same operations are done as above. Coverage is calculated and chromosome X will always be 133300518 of length.
#For positions over 133300518 the same operations are done as above. Coverage is calculated and associated to PAR chromosome; length will be always 5708626.

rm output*
done

#Files with prefix output - temporary - are removed

cp ${outfile} ${output_folder}
rm ${outfile}

#Resulting file is moved to the final folder and deleted from the working place.

Temporary file output.per-base.bed.gz has been kept for the sake of curiosity (the script removes temporary files) and looks like this:

In [4]:
zcat /cluster/work/pausch/temp_scratch/audald/analysis_test/coverage/{2}_output.per-base.bed.gz | head -10
zcat /cluster/work/pausch/temp_scratch/audald/analysis_test/coverage/{11}_output.per-base.bed.gz | tail -10

2	0	87	0
2	87	177	1
2	177	190	2
2	190	203	3
2	203	204	4
2	204	209	5
2	209	215	6
2	215	234	7
2	234	238	8
2	238	272	9

gzip: stdout: Broken pipe
11	106980240	106980444	1
11	106980444	106980536	0
11	106980536	106980605	1
11	106980605	106980643	2
11	106980643	106980680	1
11	106980680	106980681	0
11	106980681	106980710	1
11	106980710	106980715	0
11	106980715	106980836	1
11	106980836	106982474	0


An output.per-base.bed.gz is generated for each chromosome. The first column represents the chromosome number (notice that we have modified some names in the main script), the second column is the starting position, the third column is the end position and the fourth column is the coverage. This is, the bases between starting position and end position are covered according to column four in chromosome from column 1.

Here is the command used for submitting the jobs to the queue (1 job for each sample):

In [ ]:
for bam in `ls /cluster/work/pausch/inputs/bam/BTA_UCD12/*.bam | xargs -n 1 basename | awk -F".bam" '{print $1}'`
do
bsub -n 1 -W 6:00 -R "rusage[mem=6000,scratch=2500]" -J "mosdepth" -env "all, path=/cluster/work/pausch/inputs/bam/BTA_UCD12/, output_folder=/this/is/my/output/directory/,bam_id=${bam}" < /cluster/work/pausch/to_share/get_coverage.sh
echo $bam
fi
done

#### Step 2: get average coverage for all samples

Once the coverage for all samples is retrieved, the average coverage can be summarised for all samples using this second script:

In [ ]:
echo "bam_id ITB_id autosome x y PAR MT" > $HOME/summary_coverage.txt #Summary_coverage text file is created with the header
coverage_folder=/this/is/my/output/directory/ #Where the coverage files can be found
for bam in `ls /cluster/work/pausch/inputs/bam/BTA_UCD12/*.bam | xargs -n 1 basename | awk -F".bam" '{print $1}'` #Extracting the sample IDs for the loop
do
itb_id=`cat /cluster/work/pausch/inputs/ref/BTA/individual_information.csv | awk -v bam=$bam '$1==bam' | awk '{print $3}'`
#In order to retrieve the interbull_id. Sample ID is identified in the first column. Once identified, third column is printed
autosome=`awk '{ total += $4; count++ } END { print total/count }' <(head -n 29 ${coverage_folder}/${bam}.coverage)`
#summing up the coverage for all autosomic chromosomes and printing the division by the number of chromosomes. 
x=`tail -n 4 ${coverage_folder}/${bam}.coverage  | head -n 1 | awk '{print $4}'`
#Printing the fourth column of the X chromosome (top one when printing the four last chromosomes)
par=`tail -n 3 ${coverage_folder}/${bam}.coverage  | head -n 1 | awk '{print $4}'`
#Printing the fourth column of the PAR chromosome (top one when printing the three last chromosomes)
y=`tail -n 2 ${coverage_folder}/${bam}.coverage  | head -n 1 | awk '{print $4}'`
#Printing the fourth column of the Y chromosome (top one when printing the two last chromosomes)
mt=`tail -n 1 ${coverage_folder}/${bam}.coverage | awk '{print $4}'`
#Printing the fourth column of the MT chromosome (last chromosome)
echo $bam $itb_id $autosome $x $y $par $mt >> $HOME/summary_coverage.txt
#printing all the values in the summary_coverage file
done

Although this code is perfectly fine for the BAM files aligned to UCD, the MIT chromosome cannot be retrieved from the BAMs aligned to Angus assembly, as there is no MT-sequence in the Angus reference. Therefore, the following modified script is provided for Angus-BAM files:

In [ ]:
echo "bam_id ITB_id autosome x y PAR" > $HOME/summary_coverage.txt
coverage_folder=/this/is/my/output/directory/
for bam in `ls /cluster/work/pausch/inputs/bam/BTA_UCD12/*.bam | xargs -n 1 basename | awk -F".bam" '{print $1}'`
do
itb_id=`cat /cluster/work/pausch/inputs/ref/BTA/individual_information.csv | awk -v bam=$bam '$1==bam' | awk '{print $3}'`
autosome=`awk '{ total += $4; count++ } END { print total/count }' <(head -n 29 ${coverage_folder}/${bam}.coverage)`
x=`tail -n 3 ${coverage_folder}/${bam}.coverage  | head -n 1 | awk '{print $4}'`
par=`tail -n 2 ${coverage_folder}/${bam}.coverage  | head -n 1 | awk '{print $4}'`
y=`tail -n 1 ${coverage_folder}/${bam}.coverage  | head -n 1 | awk '{print $4}'`
echo $bam $itb_id $autosome $x $y $par  >> $HOME/summary_coverage.txt
done